In [119]:
import warnings
warnings.filterwarnings('ignore')
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

In [98]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /Applications/miniconda3/envs/env1_fyp/etc/jupyter
    - Validating...
      voila 0.3.5 OK


In [120]:
from qiskit import QuantumCircuit, Aer, assemble, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from math import sqrt, pi
from numpy.random import randint
import numpy as np
print("Imports Successful")

Imports Successful


In [121]:
N = 0
ciphertext = ""
alice_bases = ""
bob_bases = ""
eve_bases = ""
eve_message = ""
measured_message = ""
clean_results = ""
alice_same_bases = ""
bob_same_bases = ""
alice_key = []
bob_key = []

In [124]:
#Now we are going to code the encoding function. This is the server which is going to entangle the qubits.
def encode_bits(number_of_bits): #The parameter denotes the number of bits that need to be used
    ciphertext = []
    for i in range(number_of_bits):
        quantum_circuit = QuantumCircuit(2, 4)
        quantum_circuit.h(0)
        quantum_circuit.cx(0, 1)
        quantum_circuit.barrier()
        ciphertext.append(quantum_circuit)
    return ciphertext

In [127]:
def decode_qubits(ciphertext, bases1, bases2):
    sim = Aer.get_backend('aer_simulator')
    message = []
    for i in range(len(bases1)):
        if bases1[i] == 0: #Measurement in the z-axis
            ciphertext[i].measure(0, 0)
        else: #Measurement in the x-axis
            ciphertext[i].h(0)
            ciphertext[i].measure(0, 0)
        if bases2[i] == 0: #Measurement in the z-axis
            ciphertext[i].measure(1, 1)
        else: #Measurement in the x-axis
            ciphertext[i].h(1)
            ciphertext[i].measure(1, 1)
        qobj = assemble(ciphertext[i])
        result = sim.run(qobj).result()
        message.append(list(result.get_counts(ciphertext[i]).keys())[0])
    return message

In [128]:
def interception(ciphertext, bases):
    message = []
    sim_1 = Aer.get_backend('aer_simulator')
    for i in range(len(bases)):
        if bases[i] == 0:
            ciphertext[i].measure(0, 2)
        else:
            ciphertext[i].h(0)
            ciphertext[i].measure(0, 2)
        qobj = assemble(ciphertext[i])
        result = sim_1.run(qobj).result()
        message.append(list(result.get_counts(ciphertext[i]).keys())[0])
    return message

In [129]:
def remove_garbage(a_bases, b_bases, bits):
    alice_same_bases = []
    bob_same_bases = []
    new_bits = []
    for i in range(len(b_bases)):
        if a_bases[i] == b_bases[i]:
            alice_same_bases.append(a_bases[i])
            bob_same_bases.append(b_bases[i])
            new_bits.append(bits[i])
        else:
            pass
    return new_bits, alice_same_bases, bob_same_bases

In [130]:
# Image Widget

file = open("System.jpg", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='500',
                    height = '500',
                    align = 'center'
                )

label_headline = widgets.Label(
                    value='Photo by CDC on Unsplash',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [131]:
grand_val = widgets.ToggleButtons(
            options=['Without Interception', 'With Interception']
        )
#widgets.HBox([widgets.Label('Toggle between simulation with interception and simulation without interception:'), grand_val])

In [132]:
# number of friends

N = widgets.IntSlider(
            value=3, # default value
            min=0, 
            max=500,
            step=1,
            style={'description_width': 'initial', 'handle_color': '#16a085'} 
        )
#widgets.HBox([widgets.Label('Select the number of bits for encoding: '), N])

In [133]:
# button send

button_start_simulation = widgets.Button(
                description='start_simulation',
                tooltip='Start Simulation',
                style={'description_width': 'initial'}
            )
output = widgets.Output()

def start_simulation_event(event):
    with output:
        clear_output()
        print("The simulation has started.")
        time.sleep(2)
        if grand_val.value == "Without Interception":
            np.random.seed(seed = 0)
            print("Generating Random Bases for Alice: ")
            alice_bases = randint(2, size = N.value)
            print(alice_bases)
            time.sleep(2)
            print("Generating Random Bases for Bob: ")
            bob_bases = randint(2, size = N.value)
            print(bob_bases)
            print("Now the central server will entangle qubits and send to Alice and Bob: ")
            ciphertext = encode_bits(N.value)
            time.sleep(2)
            print("Now Alice and Bob will measure their respective qubits.")
            measured_message = decode_qubits(ciphertext, alice_bases, bob_bases)
            time.sleep(2)
            print("Now Alice and Bob will publicly declare their measurement bases for the other person to clean their output: ")
            print("Alice Bases: ", alice_bases)
            print("Bob Bases: ", bob_bases)
            time.sleep(2)
            clean_results, alice_same_bases, bob_same_bases = remove_garbage(alice_bases, bob_bases, measured_message)
            print("Now Alice and Bob will each print a part of their keys and check if anyone has been eavesdropping.")
            print(clean_results[:30])
            print("Alice's Bases: ", alice_same_bases[:30])
            print("Bob's Bases: ", bob_same_bases[:30])
            #ans = input("Generate Keys for Alice and Bob ? (y/n)")
            alice_key = []
            bob_key = []
            for measurement in clean_results:
                alice_key.append(measurement[3])
                bob_key.append(measurement[2])
            alice_key = "".join(alice_key)
            bob_key = "".join(bob_key)
            print(alice_key)
            print(bob_key)
            print("This is the end of the simulation.")
        else:
            np.random.seed(seed = 0)
            print("Generating Random Bases for Alice: ")
            alice_bases = randint(2, size = N.value)
            time.sleep(2)
            print(alice_bases)
            print("Generating Random Bases for Bob: ")
            bob_bases = randint(2, size = N.value)
            time.sleep(2)
            print(bob_bases)
            print("Generating Random Bases for Eve: ")
            eve_bases = randint(2, size = N.value)
            time.sleep(2)
            print(eve_bases)
            print("Now the central server will entangle qubits and send to Alice and Bob: ")
            ciphertext = encode_bits(N.value)
            time.sleep(2)
            print("However, Eve intercepts the entangled qubit which was supposed to be send to Bob")
            print("To get an output, she measures the entangled qubit, effectively breaking the entanglement.")
            eve_measurement = interception(ciphertext, eve_bases)
            print("Now Alice and Bob will measure their respective qubits.")
            measured_message = decode_qubits(ciphertext, alice_bases, bob_bases)
            time.sleep(2)
            print("Now Alice and Bob will publicly declare their measurement bases for the other person to clean their output: ")
            print("Alice Bases: ", alice_bases)
            print("Bob Bases: ", bob_bases)
            time.sleep(2)
            clean_results, alice_same_bases, bob_same_bases = remove_garbage(alice_bases, bob_bases, measured_message)
            print("Now Alice and Bob will each print a part of their keys and check if anyone has been eavesdropping.")
            print(clean_results[:30])
            print("Alice's Bases: ", alice_same_bases[:30])
            print("Bob's Bases: ", bob_same_bases[:30])
            #ans = input("Generate Keys for Alice and Bob ? (y/n)")
            alice_key = []
            bob_key = []
            for measurement in clean_results:
                alice_key.append(measurement[3])
                bob_key.append(measurement[2])
            alice_key = "".join(alice_key)
            bob_key = "".join(bob_key)
            print(alice_key)
            print(bob_key)
            print("This is the end of the simulation.")
            
        #print("Sent message: ")
        #print(f"Dear Grandma! This is your favourite {grand.value}, {name.value}.")
        #print(f"I would love to come over on {date.value} for dinner, if that's okay for you!")
        #print(f"Also, if you don't mind, I'll bring along {friends.value} hungry ghosts for your delicious food!")

        
button_start_simulation.on_click(start_simulation_event)
vbox_result = widgets.VBox([button_start_simulation, output])

In [134]:
# stacked right hand side

text_0 = widgets.HTML(value="<h1>Quantum Computing for Secured Communication!</h1>")
text_1 = widgets.HTML(value="<h2>Choose the number of bits for encoding of bits: </h2>")
text_2= widgets.HTML(value="<h2>Choose if you want to see the simulation with Interception or without Interception: </h2>")
text_3= widgets.HTML(value="<h2>Press the button below to start the simulation: </h2>")
text_4= widgets.HTML(value="<h2>Also, if you don't mind, I'll bring along </h2>")
text_5= widgets.HTML(value="<h2>hungry ghosts for dinner, if that's okay for you!</h2>")

vbox_text = widgets.VBox([text_0, text_1, N,text_2, grand_val,text_3,  vbox_result])

In [116]:
page = widgets.VBox([vbox_headline, vbox_text])
display(page)

In [135]:
!pip freeze > requirements.txt